# Extract information from HDF5 files and export to .csv file

## This dataset and extraction file of the HDF5 files, are courtesy of the Million Song Dataset created for machine learning.  For more information, please visit their site at https://labrosa.ee.columbia.edu/millionsong/

Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, and Paul Lamere. 
The Million Song Dataset. In Proceedings of the 12th International Society
for Music Information Retrieval Conference (ISMIR 2011), 2011.

In [25]:
import os
import sys
import time
import glob
import tables
import scipy.io as sio
import pandas as pd
import numpy as np
import hdf5_getters
from pandas import read_hdf
import numpy as np

In [2]:
def get_all_files(basedir,ext='.h5') :
    """
    From a root directory, go through all subdirectories
    and find all files with the given extension.
    Return all absolute paths in a list.
    """
    allfiles = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files :
            allfiles.append( os.path.abspath(f) )
    return allfiles

In [3]:
#this uses the hdff_get_headers.py file to extract the data from the MSD.  Takes about 15 minutes to run

def transfer(h5path,matpath=None,force=False):
    """
    Transfer an HDF5 song file (.h5) to a matfile (.mat)
    If there are more than one song in the HDF5 file, each
    field name gets a number happened: 1, 2, 3, ...., numfiles
    PARAM
        h5path  - path to the HDF5 song file
        matpath - path to the new matfile, same as HDF5 path
                  with a different extension by default
        force   - if True and matfile exists, overwrite
    RETURN
        True if the file was transfered, False if there was
        a problem.
        Could also raise an IOException
    NOTE
        All the data has to be loaded in memory! be careful
        if one file contains tons of songs!
    """
    # sanity checks
    if not os.path.isfile(h5path):
        print ('path to HF5 files does not exist:',h5path)
        return False
    if not os.path.splitext(h5path)[1] == '.h5':
        print ('expecting a .h5 extension for file:',h5path)
        return False
    # check matfile
    if matpath is None:
        matpath = os.path.splitext(h5path)[0] + '.mat'
    if os.path.exists(matpath):
        if not force: 
            print('matfile',matpath,'already exists (delete or force):')
            return False
    # get all getters! we assume that all we need is in hdf5_getters.py
    # further assume that they have the form get_xxxxxx and that's the
    # only thing that has that form
    getters = list(filter(lambda x: x[:4] == 'get_', hdf5_getters.__dict__.keys()))
    getters.remove("get_num_songs") # special case
    # open h5 file
    h5 = hdf5_getters.open_h5_file_read(h5path)
    # transfer
    nSongs = hdf5_getters.get_num_songs(h5)
    matdata = {'transfer_note':'transferred on '+time.ctime()+' from file: '+h5path}
    try:
        # iterate over songs
        for songidx in range(nSongs):
            # iterate over getter
            for getter in getters:
                gettername = getter[4:]
                if nSongs > 1:
                    gettername += str(songidx+1)
                data = hdf5_getters.__getattribute__(getter)(h5,songidx)
                matdata[gettername] = data
    except MemoryError:
        print('Error')
        raise
    finally:
        # close h5
        h5.close()
    # create
    return matdata

In [4]:
h5s = get_all_files('/Users/samlundberg/desktop/capstonems/MillionSongSubset/data/')

In [5]:
df = pd.DataFrame()

In [6]:
%%time
for file in h5s:
    xd = transfer(file)
    df = df.append(pd.Series(xd), ignore_index=True)

CPU times: user 13min 40s, sys: 31 s, total: 14min 11s
Wall time: 15min 7s


In [7]:
dfcopy=df.copy()

In [8]:
#df=dfcopy.copy()

## Before saving to a .csv where the formatting changes, all of the song attrtibutes need to be converted from a list of values, to a mean of those values for further use in the modeling process later on

In [9]:
#this is taking the list of numbers for each audio measurement and finds the average to use later on
#the proper way would be to create a separate field for each value, but that would require a great deal more time 
#and subject matter expertese

#df['artist_terms_1'] = df['artist_terms'].apply(lambda col: np.mean(col))
df['artist_terms_freq_mean'] = df['artist_terms_freq'].apply(lambda col: np.mean(col))
df['artist_terms_weight_mean'] = df['artist_terms_weight'].apply(lambda col: np.mean(col))
df['segments_start_mean'] = df['segments_start'].apply(lambda col: np.mean(col))
df['segments_pitches_mean'] = df['segments_pitches'].apply(lambda col: np.mean(col))
df['segments_timbre_mean'] = df['segments_timbre'].apply(lambda col: np.mean(col))
df['segments_loudness_max_mean'] = df['segments_loudness_max'].apply(lambda col: np.mean(col))
df['segments_loudness_max_time_mean'] = df['segments_loudness_max_time'].apply(lambda col: np.mean(col))
df['segments_loudness_start_mean'] = df['segments_loudness_start'].apply(lambda col: np.mean(col))
df['sections_start_mean'] = df['sections_start'].apply(lambda col: np.mean(col))
df['sections_confidence_mean'] = df['sections_confidence'].apply(lambda col: np.mean(col))
df['beats_start_mean'] = df['beats_start'].apply(lambda col: np.mean(col))
df['beats_confidence_mean'] = df['beats_confidence'].apply(lambda col: np.mean(col))
df['bars_start_mean'] = df['bars_start'].apply(lambda col: np.mean(col))
df['bars_confidence_mean'] = df['bars_confidence'].apply(lambda col: np.mean(col))
df['tatums_start_mean'] = df['tatums_start'].apply(lambda col: np.mean(col))
df['tatums_confidence_mean'] = df['tatums_confidence'].apply(lambda col: np.mean(col))

/Users/samlundberg/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [10]:
df.head()

,analysis_sample_rate,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_mbtags,...,segments_loudness_max_time_mean,segments_loudness_start_mean,sections_start_mean,sections_confidence_mean,beats_start_mean,beats_confidence_mean,bars_start_mean,bars_confidence_mean,tatums_start_mean,tatums_confidence_mean
0,22050.0,16971.0,0.557460,0.386152,b'AREJXK41187B9A4ACC',46.71067,b'France',1.71819,b'c43bb0d6-94d7-410f-80fb-e5a243b18d23',[],...,0.052350,-19.229238,57.679828,0.591333,71.025470,0.647949,70.298204,0.136000,71.025247,0.322668
1,22050.0,92108.0,0.626958,0.434860,b'AR2XRFQ1187FB417FE',NaN,b'',NaN,b'a69cd724-2f57-4ed0-bfed-ba20401eb84c',[],...,0.086557,-21.786326,119.968496,0.534778,123.794350,0.346571,130.630938,0.163623,123.792848,0.089049
2,22050.0,1701.0,0.425724,0.000000,b'ARODOO01187FB44F4A',NaN,b'',NaN,b'60bd8a1c-c093-4849-8f28-08101ca059b1',[],...,0.097573,-33.395346,21.487460,0.323250,33.789365,0.594763,30.497964,0.110357,33.788618,0.222222
3,22050.0,92184.0,0.611495,0.334520,b'ARJGW911187FB586CA',NaN,b'',NaN,b'44b5b950-2ae2-403a-8c67-82d8fc72033d',[],...,0.054662,-10.498846,48.594217,0.495333,79.666289,0.303867,79.627999,0.148167,79.796557,0.093216
4,22050.0,278655.0,0.367255,0.311616,b'AR9HQ6Y1187FB3C2CB',NaN,b'',NaN,b'0e6524bd-6641-46a6-bce5-96f06c19aa46',[],...,0.101949,-26.980752,84.804912,0.625700,98.247921,0.356867,97.253457,0.124979,98.247266,0.105013


In [11]:
#set paramater values
a,b,c,d=[df['artist_name'],df['artist_terms'] ,df['artist_terms_freq'],df['artist_terms_weight']]

a[0],b[0][0],c[0][0],d[0][0]

(b'Rapha\xc3\xabl', b'chanson', 0.9106976156829982, 1.0)

In [12]:
a[0],b[0][1],c[0][1],d[0][1]

(b'Rapha\xc3\xabl', b'visual kei', 0.892165696549715, 0.9915376816967095)

In [13]:
a[0][1],b[0][1],c[0][1]

(97, b'visual kei', 0.892165696549715)

In [14]:
a[1],b[1][0],c[1][0],d[1][0]

(b'Julie Zenatti', b'chanson', 1.0, 1.0)

In [15]:
b[1]

array([b'chanson', b'dance pop', b'pop rock', b'soft rock',
       b'female vocalist', b'pop', b'french pop', b'french',
       b'chanson francaise', b'female', b'rock', b'canadian', b'soul',
       b'francophone', b'french female', b'singer', b'artiste francais',
       b'fr', b'sad'], dtype='|S256')

In [16]:
#drop rows with a blank list [] in artist_terms so there will be a genre label for every artist
df=df[df['artist_terms'].map(lambda i: len(i)) > 0]

In [17]:
#find list of genres 
i=0
list_g=[]

acount=0
count=0
for j in range((a.shape[0])):
    for i in range((b[j].shape[0])):
        if len(b[j]) != 0:
        #print(b[i][0])

            genre=b[j][i]
            list_g.append(genre)
            count=(count + 1)
            i= count

    #print(list_g)        


In [39]:
#there are a huge number of genres being tracked, over 3500
#list_g

In [19]:
#create unique values of genre (remove duplicates)
g=[el for i, el in enumerate(list_g) if el not in list_g[:i]]

In [42]:
#this creates a genre column in the df and uses the top genre value as the defining genre for that artist

df['genre']=""
list_a=[]

j=0
for j in range((a.shape[0])):
    if len(b[j]) != 0:
        i=0

        #for i in range((b[j].shape[0])):
        #all=[a[j],b[j][0]]
        
        #all=b[j][0]
        #list_a.append(all)

        #print(all)
        #df['genre']=pd.Series(list_a)
        df['genre'][j]=b[j][0]

/Users/samlundberg/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [43]:
#reset index numbers
df.reset_index(drop=True, inplace=True)

In [44]:
df.head()

,analysis_sample_rate,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_mbtags,...,segments_loudness_start_mean,sections_start_mean,sections_confidence_mean,beats_start_mean,beats_confidence_mean,bars_start_mean,bars_confidence_mean,tatums_start_mean,tatums_confidence_mean,genre
0,22050.0,16971.0,0.557460,0.386152,b'AREJXK41187B9A4ACC',46.71067,b'France',1.71819,b'c43bb0d6-94d7-410f-80fb-e5a243b18d23',[],...,-19.229238,57.679828,0.591333,71.025470,0.647949,70.298204,0.136000,71.025247,0.322668,b'chanson'
1,22050.0,92108.0,0.626958,0.434860,b'AR2XRFQ1187FB417FE',NaN,b'',NaN,b'a69cd724-2f57-4ed0-bfed-ba20401eb84c',[],...,-21.786326,119.968496,0.534778,123.794350,0.346571,130.630938,0.163623,123.792848,0.089049,b'chanson'
2,22050.0,1701.0,0.425724,0.000000,b'ARODOO01187FB44F4A',NaN,b'',NaN,b'60bd8a1c-c093-4849-8f28-08101ca059b1',[],...,-33.395346,21.487460,0.323250,33.789365,0.594763,30.497964,0.110357,33.788618,0.222222,b'early music'
3,22050.0,92184.0,0.611495,0.334520,b'ARJGW911187FB586CA',NaN,b'',NaN,b'44b5b950-2ae2-403a-8c67-82d8fc72033d',[],...,-10.498846,48.594217,0.495333,79.666289,0.303867,79.627999,0.148167,79.796557,0.093216,b'post-hardcore'
4,22050.0,278655.0,0.367255,0.311616,b'AR9HQ6Y1187FB3C2CB',NaN,b'',NaN,b'0e6524bd-6641-46a6-bce5-96f06c19aa46',[],...,-26.980752,84.804912,0.625700,98.247921,0.356867,97.253457,0.124979,98.247266,0.105013,b'orchestra'


In [45]:
#create decade column

df['year']=df['year'].astype(int)

# NEAREST 10 FOR DECADE START
df['decade_start'] = (df['year'] // 10) * 10 + 1

# ADJUST FOR YEARS ENDING IN ZERO
df.loc[(df['year'] % 10) == 0, 'decade_start'] = df['decade_start'] - 10

# CALCULATE DECADE RANGE
#df['decade'] = df['decade_start'].astype('str') + ' - ' + \
#                     (df['decade_start'] + 9).astype('str') 

df['decade'] = df['decade_start'].astype('int') -1

df.drop(['decade_start'], inplace=True, axis=1)


## Create genre table that lists all genres, and their related scores, for each artist

In [48]:
#del genre_alt
#create unique values of genre (remove duplicates)
g=[el for i, el in enumerate(list_g) if el not in list_g[:i]]

#create genre df with column labels as genre type and empty df
genre_alt=pd.DataFrame(g).T
genre_alt.columns=g
genre_alt=genre_alt.drop(genre_alt.index[[0]])
genre_alt.insert(loc=0, column='artist', value="")

In [49]:
#this is good-takes quite awhile to run
#attempt to just pull just first genre, score from each artist

list_a=[]
j=0

for j in range((a.shape[0])):
    if len(b[j]) != 0:
        i=0

        #all=[a[j],b[j][0],c[j][0] * d[j][0]]
        genre_val=c[j][0] * d[j][0]
        list_a.append(all)

        genre_dict=({'artist':a[j],b[j][i]:genre_val})
        genre1  = pd.DataFrame(genre_dict, index=[j])
        genre_alt = pd.concat([genre_alt, genre1], sort=False)


In [50]:
genre_alt.fillna(0, inplace=True)

In [51]:
genre_alt.drop_duplicates(['artist'], inplace=True)
genre_alt = genre_alt.reset_index()

In [52]:
genre_alt

,index,artist,b'chanson',b'visual kei',b'hip hop',b'pop rock',b'british pop',b'jrock',b'world music',b'downtempo',...,b'neo new wave',b'polish rap',b'warsaw',b'brazilian popular singers',b'riot',b'80s hardcore punk',b'ussr',b'urdu',b'progressive indie',b'psycho punk'
0,0,b'Rapha\xc3\xabl',0.910698,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,b'Julie Zenatti',1.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,b'The Baltimore Consort',0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3,b'I Hate Sally',0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4,b'Orlando Pops Orchestra',0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5,5,b'Brand X',0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
6,6,b'Amorphis',0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,7,b'Inoki',0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
8,8,b'Dead Kennedys',0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
9,9,b'Brigada Victor Jara',0.000000,0.0,0.0,0.0,0.000000,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
u2=genre[genre['artist']== b'U2'].T

#U2 has 37 genre labels...this is problematic and probalby won't be used in this format
u2[1117].value_counts()

TypeError: byte indices must be integers or slices, not str

In [ ]:
#alt method to populate genre table with values-not as efficient

In [54]:
#del genre
#create unique values of genre (remove duplicates)
g=[el for i, el in enumerate(list_g) if el not in list_g[:i]]

#create genre df with column labels as genre type and empty df
genre=pd.DataFrame(g).T
genre.columns=g
genre=genre.drop(genre.index[[0]])
genre.insert(loc=0, column='artist', value="")

In [55]:
#genrecopy=genre.copy()
#genre=genrecopy.copy()

In [56]:
test=df.copy()

In [57]:
#set a,b,c,d values for loop below to find genres and respecitve values
a,b,c,d=[test['artist_name'],test['artist_terms'] ,test['artist_terms_freq'],test['artist_terms_weight']]

a[0][0],b[0][0],c[0][0],d[0][0]

(82, b'chanson', 0.9106976156829982, 1.0)

In [58]:
#creates master genre table for each genre for each artist
#takes a LONG time to run

list_gen={}
art_dict={}


for j in range((a.shape[0])):
    list_gen={}
    art_dict={}
    art_dict={'artist':a[j]}

    for i in range((b[j].shape[0])):
        genre_val=c[j][i] * d[j][i]
        genre_dict={b[j][i]:genre_val}
        list_gen.update(genre_dict)

    art_dict.update(list_gen)
    genre1  = pd.DataFrame(art_dict, index=[j])
    genre = pd.concat([genre, genre1], sort=False)      

In [59]:
#fill NaN values with 0
genre.fillna(0, inplace=True)

In [60]:
genre.head()

,artist,b'chanson',b'visual kei',b'hip hop',b'pop rock',b'british pop',b'jrock',b'world music',b'downtempo',b'singer-songwriter',...,b'neo new wave',b'polish rap',b'warsaw',b'brazilian popular singers',b'riot',b'80s hardcore punk',b'ussr',b'urdu',b'progressive indie',b'psycho punk'
0,b'Rapha\xc3\xabl',0.910698,0.884616,0.95546,0.893201,0.771696,0.752162,0.704693,0.686596,0.715531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,b'Julie Zenatti',1.000000,0.000000,0.00000,0.896392,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,b'The Baltimore Consort',0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,b'I Hate Sally',0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,b'Orlando Pops Orchestra',0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
#drop duplicate rows of artist genre values
genre.drop_duplicates(['artist'], inplace=True)
genre = genre.reset_index()

In [62]:
#export df to csv file
df.to_csv('./data/million_songs.csv')

In [63]:
#export genre table to csv
genre.to_csv('./data/genre.csv')

In [64]:
#export genre_alt table to csv
genre_alt.to_csv('./data/genrealt.csv')